In [1]:
# We will be using the adjusted cloud radiative effect method for determining cloud feedbacks
# All-sky and clear-sky feedbacks are made in Albedo/Q/T_feedbacks.ipynb
# ERF for 4xCO2 is made in gregory_method_ERF.ipynb

# By: Ty Janoski
# Edited: 10.15.21

In [2]:
# import statements

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xesmf as xe

%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [3]:
# create function for taking spatial averages, while weighting for latitude
def spatial_mean(ds_in, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    zonal = ds_in.mean(dim='lon').sel(lat=slice(lat_bound_s,lat_bound_n))
    weights = np.cos(np.deg2rad(zonal.lat)) / np.sum(np.cos(np.deg2rad(zonal.lat)))
    return((zonal * weights).sum(dim='lat'))

In [4]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx01/janoski/cesm/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None,use_cftime=True))

## Previously, we used the all-sky and clear-sky ERF to calculate the cloud feedbacks - this is deprecated now.

In [103]:
# read in the effective radiative forcing determined using gregory method in gregory_method_ERF.ipynb
ERF_SW_as = xr.open_dataarray('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_as.nc',use_cftime=True)
ERF_SW_cs = xr.open_dataarray('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_SW_cs.nc',use_cftime=True)
ERF_LW_as = xr.open_dataarray('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_as.nc',use_cftime=True)
ERF_LW_cs = xr.open_dataarray('/dx02/janoski/cesm-LE/ERF/CESM-LE_4xCO2_ERF_LW_cs.nc',use_cftime=True)

In [104]:
# we will spatially average latitudinal sections to remove some residual noise from the gregory method
ERF_SW_as = xr.where(ERF_SW_as.lat>70,spatial_mean(ERF_SW_as,lat_bound_s=70),ERF_SW_as)
ERF_SW_as = xr.where((ERF_SW_as.lat<=70) & (ERF_SW_as.lat>30),spatial_mean(ERF_SW_as,lat_bound_s=30,lat_bound_n=70),ERF_SW_as)
ERF_SW_as = xr.where((ERF_SW_as.lat<=30) & (ERF_SW_as.lat>-30),spatial_mean(ERF_SW_as,lat_bound_s=-30,lat_bound_n=30),ERF_SW_as)
ERF_SW_as = xr.where((ERF_SW_as.lat<=-30) & (ERF_SW_as.lat>-70),spatial_mean(ERF_SW_as,lat_bound_s=-70,lat_bound_n=-30),ERF_SW_as)
ERF_SW_as = xr.where((ERF_SW_as.lat<=-70),spatial_mean(ERF_SW_as,lat_bound_n=-70),ERF_SW_as)

ERF_SW_cs = xr.where(ERF_SW_cs.lat>70,spatial_mean(ERF_SW_cs,lat_bound_s=70),ERF_SW_cs)
ERF_SW_cs = xr.where((ERF_SW_cs.lat<=70) & (ERF_SW_cs.lat>30),spatial_mean(ERF_SW_cs,lat_bound_s=30,lat_bound_n=70),ERF_SW_cs)
ERF_SW_cs = xr.where((ERF_SW_cs.lat<=30) & (ERF_SW_cs.lat>-30),spatial_mean(ERF_SW_cs,lat_bound_s=-30,lat_bound_n=30),ERF_SW_cs)
ERF_SW_cs = xr.where((ERF_SW_cs.lat<=-30) & (ERF_SW_cs.lat>-70),spatial_mean(ERF_SW_cs,lat_bound_s=-70,lat_bound_n=-30),ERF_SW_cs)
ERF_SW_cs = xr.where((ERF_SW_cs.lat<=-70),spatial_mean(ERF_SW_cs,lat_bound_n=-70),ERF_SW_cs)


In [105]:
# we will spatially average latitudinal sections to remove some residual noise from the gregory method
ERF_LW_as = xr.where(ERF_LW_as.lat>70,spatial_mean(ERF_LW_as,lat_bound_s=70),ERF_LW_as)
ERF_LW_as = xr.where((ERF_LW_as.lat<=70) & (ERF_LW_as.lat>30),spatial_mean(ERF_LW_as,lat_bound_s=30,lat_bound_n=70),ERF_LW_as)
ERF_LW_as = xr.where((ERF_LW_as.lat<=30) & (ERF_LW_as.lat>-30),spatial_mean(ERF_LW_as,lat_bound_s=-30,lat_bound_n=30),ERF_LW_as)
ERF_LW_as = xr.where((ERF_LW_as.lat<=-30) & (ERF_LW_as.lat>-70),spatial_mean(ERF_LW_as,lat_bound_s=-70,lat_bound_n=-30),ERF_LW_as)
ERF_LW_as = xr.where((ERF_LW_as.lat<=-70),spatial_mean(ERF_LW_as,lat_bound_n=-70),ERF_LW_as)

ERF_LW_cs = xr.where(ERF_LW_cs.lat>70,spatial_mean(ERF_LW_cs,lat_bound_s=70),ERF_LW_cs)
ERF_LW_cs = xr.where((ERF_LW_cs.lat<=70) & (ERF_LW_cs.lat>30),spatial_mean(ERF_LW_cs,lat_bound_s=30,lat_bound_n=70),ERF_LW_cs)
ERF_LW_cs = xr.where((ERF_LW_cs.lat<=30) & (ERF_LW_cs.lat>-30),spatial_mean(ERF_LW_cs,lat_bound_s=-30,lat_bound_n=30),ERF_LW_cs)
ERF_LW_cs = xr.where((ERF_LW_cs.lat<=-30) & (ERF_LW_cs.lat>-70),spatial_mean(ERF_LW_cs,lat_bound_s=-70,lat_bound_n=-30),ERF_LW_cs)
ERF_LW_cs = xr.where((ERF_LW_cs.lat<=-70),spatial_mean(ERF_LW_cs,lat_bound_n=-70),ERF_LW_cs)

dERF_LW = ERF_LW_cs - ERF_LW_as
dERF_SW = ERF_SW_cs - ERF_SW_as

## Instead, we are going to use the CAM4 4xCO2 stratospheric-adjusted radiative forcing calculated using PORT.

In [ ]:
# read in data file
fin = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(fin, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
dRFLW_TOA = -1 * (regridder(fin.FLNTC) - regridder(fin.FLNT))
dRFSW_TOA = regridder(fin.FSNTC) - regridder(fin.FSNT)
dRFLW_SFC = -1 * (regridder(fin.FLNSC) - regridder(fin.FLNS))
dRFSW_SFC = regridder(fin.FSNSC) - regridder(fin.FSNS)

In [ ]:
#### note: convention at TOA is that positive = downward radiative flux (so negative values = cooling)
for e in range(1,101,1):
    if(e%5==0):
        print(e)
    # read in T feedbacks
    lapse = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_lapse_rad_perturbs.nc',
                            use_cftime=True)
    planck = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Planck_rad_perturbs.nc',
                             use_cftime=True)
    dT = (lapse.FLNTC - lapse.FLNT) + (planck.FLNTC - planck.FLNT)

    # Q feedbacks
    Q = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc',
                        use_cftime=True)
    dQ_LW = Q.FLNTC - Q.FLNT
    dQ_SW = Q.FSNTC - Q.FSNT

    # albedo
    α = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc',
                        use_cftime=True)
    dα = (α.FSNTC - α.FSNT).fillna(0)

    m=1
    
    # calc change in CRE
    FLNT = read_in('ctrl',m,e,'FLNT').FLNT
    FLNTC = read_in('ctrl',m,e,'FLNTC').FLNTC
    FSNT = read_in('ctrl',m,e,'FSNT').FSNT
    FSNTC = read_in('ctrl',m,e,'FSNTC').FSNTC

    CRE_ctrl_SW = (FSNT - FSNTC) # it's SW - LW because LW is positive upwards (we want it positive downwards)
    CRE_ctrl_LW = -1*(FLNT - FLNTC)

    FLNT = read_in('4xCO2',m,e,'FLNT').FLNT
    FLNTC = read_in('4xCO2',m,e,'FLNTC').FLNTC
    FSNT = read_in('4xCO2',m,e,'FSNT').FSNT
    FSNTC = read_in('4xCO2',m,e,'FSNTC').FSNTC

    CRE_exp_SW = (FSNT - FSNTC)
    CRE_exp_LW = -1*(FLNT - FLNTC)

    dCRE_SW = CRE_exp_SW - CRE_ctrl_SW
    dCRE_LW = CRE_exp_LW - CRE_ctrl_LW

    # basis for adjustment method is to adjust the change in CRE for environmental masking of other radiative perturbations
    # albedo terms has some nans, so just turn them into 0s
#     cloud_SW = dCRE_SW + dQ_SW + dα +  dERF_SW
#     cloud_LW = dCRE_LW + dQ_LW + dT + dERF_LW
    cloud_SW = dCRE_SW + dQ_SW + dα + dRFSW_TOA
    cloud_LW = dCRE_LW + dQ_LW + dT + dRFLW_TOA
    
#     out_SW = xr.merge([cloud_SW.rename('cloud_SW'),dCRE_SW.rename('dCRE_SW'),dQ_SW.rename('dQ_SW'),dα.rename('dα'),dERF_SW.rename('dERF_SW')])
#     out_LW = xr.merge([cloud_LW.rename('cloud_LW'),dCRE_LW.rename('dCRE_LW'),dQ_LW.rename('dQ_LW'),dT.rename('dT'),dERF_LW.rename('dERF_LW')])
    
    out_SW = xr.merge([cloud_SW.rename('cloud_SW'),dCRE_SW.rename('dCRE_SW'),dQ_SW.rename('dQ_SW'),dα.rename('dα'),dRFSW_TOA.rename('dRFSW_TOA')])
    out_LW = xr.merge([cloud_LW.rename('cloud_LW'),dCRE_LW.rename('dCRE_LW'),dQ_LW.rename('dQ_LW'),dT.rename('dT'),dRFLW_TOA.rename('dRFLW_TOA')])
    
    pathout_SW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM4.nc'
    pathout_LW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM4.nc'
    
    out_SW.to_netcdf(pathout_SW)
    out_LW.to_netcdf(pathout_LW)

## We are going to calculate the cloud feedbacks from a surface perspective

In [ ]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on january of a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [ ]:
#### note: convention at sfc is that positive = upwards radiative flux (so negative values = cooling)
for e in range(1,101,1):
    if(e%5==0):
        print(e)
    # read in T feedbacks
    sfc_T = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ts_rad_perturbs_sfc.nc',use_cftime=True)
    atm_T = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ta_rad_perturbs_sfc.nc',use_cftime=True)

    dT = (sfc_T.FLNSC - sfc_T.FLNS) + (atm_T.FLNSC - atm_T.FLNS)
    
#     Q feedbacks
    Q = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc',use_cftime=True)
    dQ_LW = Q.FLNSC - Q.FLNS
    dQ_SW = Q.FSNSC - Q.FSNS

#     # albedo
    α = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc',use_cftime=True)
    dα = (α.FSNSC - α.FSNS).fillna(0)

    m=1
    
#     # calc change in CRE
    FLNS = read_in('ctrl',m,e,'FLNS').FLNS
    FLNSC = read_in('ctrl',m,e,'FLNSC').FLNSC
    FSNS = read_in('ctrl',m,e,'FSNS').FSNS
    FSNSC = read_in('ctrl',m,e,'FSNSC').FSNSC

    CRE_ctrl_SW = (FSNS - FSNSC) # it's SW - LW because LW is positive upwards (we want it positive downwards)
    CRE_ctrl_LW = -1*(FLNS - FLNSC)

    FLNS = read_in('4xCO2',m,e,'FLNS').FLNS
    FLNSC = read_in('4xCO2',m,e,'FLNSC').FLNSC
    FSNS = read_in('4xCO2',m,e,'FSNS').FSNS
    FSNSC = read_in('4xCO2',m,e,'FSNSC').FSNSC

    CRE_exp_SW = (FSNS - FSNSC)
    CRE_exp_LW = -1*(FLNS - FLNSC)

    dCRE_SW = CRE_exp_SW - CRE_ctrl_SW
    dCRE_LW = CRE_exp_LW - CRE_ctrl_LW

#     # basis for adjustment method is to adjust the change in CRE for environmental masking of other radiative perturbations
#     # albedo terms has some nans, so just turn them into 0s
# #     cloud_SW = dCRE_SW + dQ_SW + dERF_SW
# #     cloud_LW = dCRE_LW + dQ_LW + dT + dERF_LW
    cloud_SW = dCRE_SW + dQ_SW + dα +  dRFSW_SFC
    cloud_LW = dCRE_LW + dQ_LW + dT + dRFLW_SFC
    
    out_SW = xr.merge([cloud_SW.rename('cloud_SW'),dCRE_SW.rename('dCRE_SW'),dQ_SW.rename('dQ_SW'),dα.rename('dα'),dRFSW_TOA.rename('dRFSW_SFC')])
    out_LW = xr.merge([cloud_LW.rename('cloud_LW'),dCRE_LW.rename('dCRE_LW'),dQ_LW.rename('dQ_LW'),dT.rename('dT'),dRFLW_TOA.rename('dRFLW_SFC')])
    
    pathout_SW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM4_sfc.nc'
    pathout_LW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM4_sfc.nc'
    
    out_SW.to_netcdf(pathout_SW)
    out_LW.to_netcdf(pathout_LW)

## Time for CAM5

In [ ]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None,use_cftime=True))

In [ ]:
# read in data file
fin = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM5_RRTMG_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(fin, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
dRFLW_TOA = -1 * (regridder(fin.FLNTC) - regridder(fin.FLNT))
dRFSW_TOA = regridder(fin.FSNTC) - regridder(fin.FSNT)
dRFLW_SFC = -1 * (regridder(fin.FLNSC) - regridder(fin.FLNS))
dRFSW_SFC = regridder(fin.FSNSC) - regridder(fin.FSNS)

In [ ]:
#### note: convention at TOA is that positive = downward radiative flux (so negative values = cooling)
for e in range(1,101,1):
    if(e%5==0):
        print(e)
    # read in T feedbacks
    lapse = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_lapse_rad_perturbs.nc',
                            use_cftime=True)
    planck = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Planck_rad_perturbs.nc',
                             use_cftime=True)
    dT = (lapse.FLNTC - lapse.FLNT) + (planck.FLNTC - planck.FLNT)

    # Q feedbacks
    Q = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc',
                        use_cftime=True)
    dQ_LW = Q.FLNTC - Q.FLNT
    dQ_SW = Q.FSNTC - Q.FSNT

    # albedo
    α = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc',
                        use_cftime=True)
    dα = (α.FSNTC - α.FSNT).fillna(0)

    m=1
    
    # calc change in CRE
    FLNT = read_in('ctrl',m,e,'FLNT').FLNT
    FLNTC = read_in('ctrl',m,e,'FLNTC').FLNTC
    FSNT = read_in('ctrl',m,e,'FSNT').FSNT
    FSNTC = read_in('ctrl',m,e,'FSNTC').FSNTC

    CRE_ctrl_SW = (FSNT - FSNTC) # it's SW - LW because LW is positive upwards (we want it positive downwards)
    CRE_ctrl_LW = -1*(FLNT - FLNTC)

    FLNT = read_in('4xCO2',m,e,'FLNT').FLNT
    FLNTC = read_in('4xCO2',m,e,'FLNTC').FLNTC
    FSNT = read_in('4xCO2',m,e,'FSNT').FSNT
    FSNTC = read_in('4xCO2',m,e,'FSNTC').FSNTC

    CRE_exp_SW = (FSNT - FSNTC)
    CRE_exp_LW = -1*(FLNT - FLNTC)

    dCRE_SW = CRE_exp_SW - CRE_ctrl_SW
    dCRE_LW = CRE_exp_LW - CRE_ctrl_LW

    # basis for adjustment method is to adjust the change in CRE for environmental masking of other radiative perturbations
    # albedo terms has some nans, so just turn them into 0s
    cloud_SW = dCRE_SW + dQ_SW + dα + dRFSW_TOA
    cloud_LW = dCRE_LW + dQ_LW + dT + dRFLW_TOA
    
    out_SW = xr.merge([cloud_SW.rename('cloud_SW'),dCRE_SW.rename('dCRE_SW'),dQ_SW.rename('dQ_SW'),dα.rename('dα'),dRFSW_TOA.rename('dRFSW_TOA')])
    out_LW = xr.merge([cloud_LW.rename('cloud_LW'),dCRE_LW.rename('dCRE_LW'),dQ_LW.rename('dQ_LW'),dT.rename('dT'),dRFLW_TOA.rename('dRFLW_TOA')])
    
    pathout_SW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM5.nc'
    pathout_LW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM5.nc'
    
    out_SW.to_netcdf(pathout_SW)
    out_LW.to_netcdf(pathout_LW)

## CAM5 surface

In [ ]:
# create function for reading in cesm-LE output
# note: each ensemble member starts on january of a different year
# please keep this in mind when combining datasets
def read_in(exp,mon,ens,var):
    """
    Use xarray to read in a netCDF file.

    Keyword arguments:
    exp -- CO2 scenario
    mon -- starting month in which CO2 is altered
    ens -- ensemble number
    var -- model output variable
    """
    filein = '/dx05/janoski/d10/Arctic_Research/cesm-LE/output/b40.1850.cam5-lens.'+exp+'.'+str(
        f"{mon:02d}")+'.'+str(f"{ens:02d}")+'.h1_'+var+'.nc'
    return(xr.open_dataset(filein,chunks=None))

In [ ]:
#### note: convention at sfc is that positive = upwards radiative flux (so negative values = cooling)
for e in range(1,101,1):
    if(e%5==0):
        print(e)
    # read in T feedbacks
    sfc_T = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ts_rad_perturbs_sfc.nc',use_cftime=True)
    atm_T = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Ta_rad_perturbs_sfc.nc',use_cftime=True)

    dT = (sfc_T.FLNSC - sfc_T.FLNS) + (atm_T.FLNSC - atm_T.FLNS)
    
#     Q feedbacks
    Q = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Q_rad_perturbs.nc',use_cftime=True)
    dQ_LW = Q.FLNSC - Q.FLNS
    dQ_SW = Q.FSNSC - Q.FSNS

#     # albedo
    α = xr.open_dataset('/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_alb_rad_perturbs.nc',use_cftime=True)
    dα = (α.FSNSC - α.FSNS).fillna(0)

    m=1
    
#     # calc change in CRE
    FLNS = read_in('ctrl',m,e,'FLNS').FLNS
    FLNSC = read_in('ctrl',m,e,'FLNSC').FLNSC
    FSNS = read_in('ctrl',m,e,'FSNS').FSNS
    FSNSC = read_in('ctrl',m,e,'FSNSC').FSNSC

    CRE_ctrl_SW = (FSNS - FSNSC) # it's SW - LW because LW is positive upwards (we want it positive downwards)
    CRE_ctrl_LW = -1*(FLNS - FLNSC)

    FLNS = read_in('4xCO2',m,e,'FLNS').FLNS
    FLNSC = read_in('4xCO2',m,e,'FLNSC').FLNSC
    FSNS = read_in('4xCO2',m,e,'FSNS').FSNS
    FSNSC = read_in('4xCO2',m,e,'FSNSC').FSNSC

    CRE_exp_SW = (FSNS - FSNSC)
    CRE_exp_LW = -1*(FLNS - FLNSC)

    dCRE_SW = CRE_exp_SW - CRE_ctrl_SW
    dCRE_LW = CRE_exp_LW - CRE_ctrl_LW

#     # basis for adjustment method is to adjust the change in CRE for environmental masking of other radiative perturbations
#     # albedo terms has some nans, so just turn them into 0s
# #     cloud_SW = dCRE_SW + dQ_SW + dERF_SW
# #     cloud_LW = dCRE_LW + dQ_LW + dT + dERF_LW
    cloud_SW = dCRE_SW + dQ_SW + dα +  dRFSW_SFC
    cloud_LW = dCRE_LW + dQ_LW + dT + dRFLW_SFC
    
    out_SW = xr.merge([cloud_SW.rename('cloud_SW'),dCRE_SW.rename('dCRE_SW'),dQ_SW.rename('dQ_SW'),dα.rename('dα'),dRFSW_TOA.rename('dRFSW_SFC')])
    out_LW = xr.merge([cloud_LW.rename('cloud_LW'),dCRE_LW.rename('dCRE_LW'),dQ_LW.rename('dQ_LW'),dT.rename('dT'),dRFLW_TOA.rename('dRFLW_SFC')])
    
    pathout_SW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM5_sfc.nc'
    pathout_LW = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.'+str(
        f"{m:02d}")+'.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM5_sfc.nc'
    
    out_SW.to_netcdf(pathout_SW)
    out_LW.to_netcdf(pathout_LW)

## Cloud Feedback Plotting

In [48]:
# create function for taking spatial averages, while weighting for latitude
def gw_mean(ds_in, gw, lat_bound_s = -91, lat_bound_n = 91):
    """
    Use xarray/numpy to calculate spatial average while weighting for latitude.
    
    Keyword arguments:
    ds_in -- Dataset or DataArray to take the average of, ALREADY ZONALLY AVERAGED
    gw -- Array of guassian weights. Should only have latitude dimension.
    lat_bound_s -- float, Southern boundary of area to average
    lat_bound_n -- float, Northern boundary of area to average
    """
    return (ds_in.sel(lat=slice(lat_bound_s,lat_bound_n)) * gw.sel(lat=slice(lat_bound_s,lat_bound_n))/gw.sel(
        lat=slice(lat_bound_s,lat_bound_n)).sum(dim='lat')).sum(dim='lat')

In [ ]:
gw = xr.open_dataarray('/dx02/janoski/cesm-LE/output/cam5_gauss_weights.nc')

In [47]:
# read in CAM4 cloud feedbacks at TOA

CAM4_SW = []
CAM4_LW = []
print('CAM4')
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    # SW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM4.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_SW']
    fin['time'] = np.arange(0,730,1)
    CAM4_SW.append(fin.mean(dim='lon'))
    
    # LW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM4.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_LW']
    fin['time'] = np.arange(0,730,1)
    CAM4_LW.append(fin.mean(dim='lon'))
    
CAM4_SW = xr.concat(CAM4_SW,dim='ens')
CAM4_SW['ens'] = np.arange(1,101,1)
CAM4_LW = xr.concat(CAM4_LW,dim='ens')
CAM4_LW['ens'] = np.arange(1,101,1)

# read in CAM5 cloud feedbacks at TOA

CAM5_SW = []
CAM5_LW = []
print('CAM5')
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    # SW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_CAM5.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_SW']
    fin['time'] = np.arange(0,730,1)
    CAM5_SW.append(fin.mean(dim='lon'))
    
    # LW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_CAM5.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_LW']
    fin['time'] = np.arange(0,730,1)
    CAM5_LW.append(fin.mean(dim='lon'))
    
CAM5_SW = xr.concat(CAM5_SW,dim='ens')
CAM5_SW['ens'] = np.arange(1,101,1)
CAM5_LW = xr.concat(CAM5_LW,dim='ens')
CAM5_LW['ens'] = np.arange(1,101,1)

# read in cloud feedbacks at TOA using ERF

ERF_SW = []
ERF_LW = []
print('ERF')
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    # SW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_SW_perturb_ERF.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_SW']
    fin['time'] = np.arange(0,730,1)
    ERF_SW.append(fin.mean(dim='lon'))
    
    # LW
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_cloud_LW_perturb_ERF.nc'
    fin = xr.open_dataset(path,use_cftime=True)['cloud_LW']
    fin['time'] = np.arange(0,730,1)
    ERF_LW.append(fin.mean(dim='lon'))
    
ERF_SW = xr.concat(ERF_SW,dim='ens')
ERF_SW['ens'] = np.arange(1,101,1)
ERF_LW = xr.concat(ERF_LW,dim='ens')
ERF_LW['ens'] = np.arange(1,101,1)

CAM4
10
20
30
40
50
60
70
80
90
100
CAM5
10
20
30
40
50
60
70
80
90
100
ERF
10
20
30
40
50
60
70
80
90
100


In [50]:
print('dTS')
dTS = []
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_dTS.nc'
    fin = xr.open_dataarray(path,use_cftime=True)
    fin['time'] = np.arange(0,730,1)
    dTS.append(fin.mean(dim='lon'))
    
dTS = xr.concat(dTS,dim='ens')
dTS['ens'] = np.arange(1,101,1)

# planck
print('Planck')
plk = []
for e in range(1,101,1):
    if(e%10==0):
        print(e)
    path = '/dx05/janoski/d10/Arctic_Research/cesm-LE/vert_int_feedbacks/b40.1850.cam5-lens.01.'+str(f"{e:02d}")+'.h1_Planck_rad_perturbs.nc'
    fin = xr.open_dataset(path,use_cftime=True)['FLNT']
    fin['time'] = np.arange(0,730,1)
    plk.append(fin.mean(dim='lon'))
    
plk = xr.concat(plk,dim='ens')
plk['ens'] = np.arange(1,101,1)

dTS
10
20
30
40
50
60
70
80
90
100
Planck
10
20
30
40
50
60
70
80
90
100


In [57]:
norm_factor = gw_mean(plk.mean(dim='ens'),gw)/gw_mean(dTS.mean(dim='ens'),gw)

In [173]:
fig,axes = plt.subplots(nrows=3,figsize=(5,7),sharex=True,sharey=False)
xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

axes[0].plot(xs,gw_mean(CAM4_SW.mean(dim='ens'),gw),color='blue',label='CAM4')
axes[0].plot(xs,gw_mean(CAM5_SW.mean(dim='ens'),gw),color='red',label='CAM5')
axes[0].plot(xs,gw_mean(ERF_SW.mean(dim='ens'),gw),color='gray',label='ERF')
axes[0].set_ylabel('SW (W/m$^2$)',fontsize=10)
axes[0].grid()
axes[0].legend(loc='lower left',fontsize=8)

axes[1].plot(xs,gw_mean(CAM4_LW.mean(dim='ens'),gw),color='blue',label='CAM4')
axes[1].plot(xs,gw_mean(CAM5_LW.mean(dim='ens'),gw),color='red',label='CAM5')
axes[1].plot(xs,gw_mean(ERF_LW.mean(dim='ens'),gw),color='gray',label='ERF')
axes[1].grid()
axes[1].set_ylabel('LW (W/m$^2$)',fontsize=10)

axes[2].plot(xs,gw_mean((CAM4_LW + CAM4_SW).mean(dim='ens'),gw),color='blue',label='CAM4')
axes[2].plot(xs,gw_mean((CAM5_LW+CAM5_SW).mean(dim='ens'),gw),color='red',label='CAM5')
axes[2].plot(xs,gw_mean((ERF_LW+ERF_SW).mean(dim='ens'),gw),color='gray',label='ERF')
axes[2].grid()
axes[2].set_ylabel('SW + LW (W/m$^2$)',fontsize=10)
axes[2].set_xlim([1,731])
axes[2].tick_params(axis='x', rotation=45,labelsize=10)
axes[2].set_xticks(x_vals)
axes[2].set_xticklabels(x_labs)

plt.suptitle('Global average cloud feedbacks')
plt.tight_layout()
plt.show()

<Figure size 360x504 with 3 Axes>

In [174]:
fig,axes = plt.subplots(nrows=3,figsize=(5,7),sharex=True,sharey=False)
xs = np.arange(1,731,1)

first_days = np.array([1,91,182,274])

x_vals = np.concatenate((first_days,first_days+365))
x_labs = np.tile(['Jan 1','Apr 1','Jul 1','Oct 1'],
                 (2))

axes[0].plot(xs,gw_mean(CAM4_SW.mean(dim='ens'),gw,lat_bound_s=70),color='blue',label='CAM4')
axes[0].plot(xs,gw_mean(CAM5_SW.mean(dim='ens'),gw,lat_bound_s=70),color='red',label='CAM5')
axes[0].plot(xs,gw_mean(ERF_SW.mean(dim='ens'),gw,lat_bound_s=70),color='gray',label='ERF')
axes[0].set_ylabel('SW (W/m$^2$)',fontsize=10)
axes[0].grid()
axes[0].legend(loc='lower left',fontsize=8)

axes[1].plot(xs,gw_mean(CAM4_LW.mean(dim='ens'),gw,lat_bound_s=70),color='blue',label='CAM4')
axes[1].plot(xs,gw_mean(CAM5_LW.mean(dim='ens'),gw,lat_bound_s=70),color='red',label='CAM5')
axes[1].plot(xs,gw_mean(ERF_LW.mean(dim='ens'),gw,lat_bound_s=70),color='gray',label='ERF')
axes[1].grid()
axes[1].set_ylabel('LW (W/m$^2$)',fontsize=10)

axes[2].plot(xs,gw_mean((CAM4_LW + CAM4_SW).mean(dim='ens'),gw,lat_bound_s=70),color='blue',label='CAM4')
axes[2].plot(xs,gw_mean((CAM5_LW+CAM5_SW).mean(dim='ens'),gw,lat_bound_s=70),color='red',label='CAM5')
axes[2].plot(xs,gw_mean((ERF_LW+ERF_SW).mean(dim='ens'),gw,lat_bound_s=70),color='gray',label='ERF')
axes[2].grid()
axes[2].set_ylabel('SW + LW (W/m$^2$)',fontsize=10)
axes[2].set_xlim([1,731])
axes[2].tick_params(axis='x', rotation=45,labelsize=10)
axes[2].set_xticks(x_vals)
axes[2].set_xticklabels(x_labs)

plt.suptitle('Arctic average cloud feedbacks')
plt.tight_layout()
plt.show()

<Figure size 360x504 with 3 Axes>

In [142]:
# read in data file
CAM4 = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM4_CAMRT_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(CAM4, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
dRFLW_TOA = -1 * (regridder(CAM4.FLNTC) - regridder(CAM4.FLNT))
dRFSW_TOA = regridder(CAM4.FSNTC) - regridder(CAM4.FSNT)
dRFLW_SFC = -1 * (regridder(CAM4.FLNSC) - regridder(CAM4.FLNS))
dRFSW_SFC = regridder(CAM4.FSNSC) - regridder(CAM4.FSNS)

# tile to make dimensions match other variables
# dRFLW_TOA = np.tile(dRFLW_TOA,(2,1,1))
# dRFSW_TOA = np.tile(dRFSW_TOA,(2,1,1))
# dRFLW_SFC = np.tile(dRFLW_SFC,(2,1,1))
# dRFSW_SFC = np.tile(dRFSW_SFC,(2,1,1))

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:478: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


In [183]:
(gw_mean(-1 * regridder(CAM4.FLNT).mean(dim='lon'),gw)).plot(color='blue',label='CAM4')
(gw_mean(-1 * regridder(CAM5.FLNT).mean(dim='lon'),gw)).plot(color='red',label='CAM5')
plt.title('Global average 4xCO2 LW RF, TOA')
plt.ylabel('RF (W/m$^2$)')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:478: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,
/home/janoski/miniconda3/envs/esmpy_env/lib/python3.7/site-packages/xesmf/frontend.py:478: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  keep_attrs=keep_attrs,


<Figure size 432x288 with 1 Axes>

In [112]:
gw_mean(dERF_SW.mean(dim='lon'),gw)

<xarray.DataArray ()>
array(-1.3490876)

In [143]:
# read in data file
CAM5 = xr.open_dataset('/dx02/janoski/cesm-LE/strat_adj_rf/CAM5_RRTMG_strat_adj_RF.nc',use_cftime=True)

# we need lat and lon from any CESM-LE file
dummy = xr.open_dataset('/dx02/janoski/cesm-LE/output/b40.1850.cam5-lens.4xCO2.01.01.h1_PS.nc')


# make dataarray to regrid too
ds_out = xr.Dataset({'lat': (['lat'], dummy.lat),
                     'lon': (['lon'], dummy.lon),
                    }
                   )

# make regridder
regridder = xe.Regridder(CAM5, ds_out, 'bilinear',periodic=True)

# get difference b/w clear-sky and all-sky fluxes
# note: longwave has to be multiplied by -1 for directional consistency
dRFLW_TOA = -1 * (regridder(CAM5.FLNTC) - regridder(CAM5.FLNT))
dRFSW_TOA = regridder(CAM5.FSNTC) - regridder(CAM5.FSNT)
dRFLW_SFC = -1 * (regridder(CAM5.FLNSC) - regridder(CAM5.FLNS))
dRFSW_SFC = regridder(CAM5.FSNSC) - regridder(CAM5.FSNS)

# tile to make dimensions match other variables
# dRFLW_TOA = np.tile(dRFLW_TOA,(2,1,1))
# dRFSW_TOA = np.tile(dRFSW_TOA,(2,1,1))
# dRFLW_SFC = np.tile(dRFLW_SFC,(2,1,1))
# dRFSW_SFC = np.tile(dRFSW_SFC,(2,1,1))
